## Identification.

• Giovana Vieira de Morais - 11921908

• Lucas Batista Gabriel - 7656147



## Name of the dataset
Dataset: City Cellular Traffic Map
URL: https://github.com/caesar0301/city-cellular-traffic-map

## Abstract
bla bla bla

## Questions to the dataset
Question 1: 

Question 2: 

Question 3: 

Question 4:

Question 5: What is the hour in the day with most users actives? 

Question 6: What's the Base Station that receives more traffic? 


## EDA
Present your EDA strategy:


In [ ]:
import random
import pandas as pd
from statistics import mean, median
import matplotlib.pyplot as plt
from matplotlib import rcParams
import folium
from folium.plugins import MarkerCluster
%matplotlib inline
rcParams['figure.figsize'] = (15, 10)

### Celular Traffic

In [ ]:
cellular_traffic_data = pd.read_csv('traceset/cellular_traffic.csv', delimiter = ',', decimal='.')

In [ ]:
cellular_traffic_data.info()

In [ ]:
cellular_traffic_data.head()

In [ ]:
cellular_traffic_data.tail()

In [ ]:
cellular_traffic_data.describe()

### Topology

In [ ]:
topology_data = pd.read_csv('traceset/topology.csv', delimiter = ',', decimal='.')

In [ ]:
topology_data.info()

In [ ]:
topology_data.head()

In [ ]:
topology_data.tail()

In [ ]:
topology_data.describe()

## Map visualization

Since we have 13269 base stations in our map, visualization can get a little difficult.
To solve that and to make computation faster, we will show our data with clusters.
Each cluster is marked with the quantity of base stations inside. Click to zoom inside that cluster.

The "swept the real longitudes and latitudes before publishing the data because we were banned by the privacy protocol in the research". So we can't visualize our topology data clearly.

In [ ]:
topology_data.count()

In [ ]:
# click to open the markers and visualize the base stations
mc = MarkerCluster()

cluster_map = folium.Map(location = [topology_data['lat'].mean(), topology_data['lon'].mean()],
                 title = 'Clustered base station')

for i in range(0, len(topology_data)):
    mc.add_child(folium.Marker([topology_data.iloc[i]['lat'], topology_data.iloc[i]['lon']]))
    
cluster_map.add_child(mc)

cluster_map

# Question 4:

# Question 5: What is the hour in the day with most users actives?

In [ ]:
cellular_traffic_data['time_hour'] = cellular_traffic_data['time_hour'].map(lambda x: pd.Timestamp(x, unit='s', tz='Asia/Shanghai'))
cellular_traffic_data['hour'] = cellular_traffic_data['time_hour'].map(lambda x: x.hour)
cellular_traffic_data['day'] = cellular_traffic_data['time_hour'].map(lambda x: x.day)

In [ ]:
total_hour_traffic = cellular_traffic_data.groupby(['hour']).mean()

In [ ]:
total_hour_traffic['users'].plot(kind='bar', title='users/hour')

We can see that 7PM has the highest mean, followed by 6PM. But it doesn't mean that the same hours have the 
the highest packets quantity as we can see below. 

In [ ]:
plt.subplot(2,2,1)
total_hour_traffic['users'].plot(kind='bar', title='users/hour')
plt.subplot(2,2,2)
total_hour_traffic['packets'].plot(kind='bar', title='packets/hour')
plt.subplot(2,2,3)
total_hour_traffic['bytes'].plot(kind='bar', title='bytes/hour')

So now we can see that the highest packet traffic occurs at 11PM and 12PM. The largest packets (the ones with more
bytes) are traveling at 2PM and 11PM.

# Question 6: What's the Base Station that receives more traffic? 
To answer this question, we have to consider some scenarios:

- What's the BS that has more users connected at the same time (per hour and per day)?
- What's the BS that has more packets (per hour and per day)?
- What's the BS that has the largest packets (per hour and per day)?

## Per day

In [ ]:
day_traffic_data = cellular_traffic_data.groupby(['bs', 'day']).mean()
day_traffic_data

In [ ]:
day_traffic_data.idxmax()

In [ ]:
max_data = day_traffic_data.idxmax()

So, as we see below, the BS with more users at the same time is **BS 7999** and the traffic peak occurred on 
August 20th, when aprox. 199 users were connected.

In [ ]:
day_traffic_data[(day_traffic_data.index.get_level_values('bs') == max_data.users[0]) &
                 (day_traffic_data.index.get_level_values('day') == max_data.users[1])]

The BS with more packets was **BS 9286**, that had aprox 162995 packets in a single day on August 26th.

In [ ]:
day_traffic_data[(day_traffic_data.index.get_level_values('bs') == max_data.packets[0]) &
                 (day_traffic_data.index.get_level_values('day') == max_data.packets[1])]

The BS with the largest packets in a hour was **BS10955** that 3.726662e+10 bytes on August 21th.

In [ ]:
day_traffic_data[(day_traffic_data.index.get_level_values('bs') == max_data.bytes[0]) &
                 (day_traffic_data.index.get_level_values('day') == max_data.bytes[1])]

In [ ]:
## plotting a map with those three max bs to check how close they are
bs_info = {
    "most_user_access": max_data.users[0],
    "most_bytes": max_data.bytes[0],
    "most_packets": max_data.packets[0]
}

bs = []
for bs_index in max_data:
    bs.append(bs_index[0])

max_day_map = folium.Map(location = [topology_data.iloc[bs]['lat'].mean(), topology_data.iloc[bs]['lon'].mean()],
                 title = 'base station with max data per day')

for i in bs_info:
    print(f"base station index of {i}: {bs_info[i]}")
    folium.Marker(location = [topology_data.iloc[bs_info[i]]['lat'], 
                              topology_data.iloc[bs_info[i]]['lon']],
                 tooltip = i).add_to(max_day_map)
    
max_day_map

In [ ]:
bs_info

In [ ]:
plt.subplot(3,1,1)
plt.title(f"users")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_bytes']]['users'].plot(label=bs_info['most_bytes'])
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_packets']]['users'].plot(label=bs_info['most_packets'])
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_user_access']]['users'].plot(label=bs_info['most_user_access'])
plt.legend()


plt.subplot(3,1,2)
plt.title(f"bytes")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_bytes']]['bytes'].plot(label=bs_info['most_bytes'])
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_packets']]['bytes'].plot(label=bs_info['most_packets'])
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_user_access']]['bytes'].plot(label=bs_info['most_user_access'])
plt.legend()

plt.subplot(3,1,3)
plt.title(f"packets")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_bytes']]['packets'].plot(label=bs_info['most_bytes'])
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_packets']]['packets'].plot(label=bs_info['most_packets'])
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_user_access']]['packets'].plot(label=bs_info['most_user_access'])

plt.legend()

plt.tight_layout()

In [ ]:
day_traffic_data[day_traffic_data.index.get_level_values('bs') == 1].corr()

In [ ]:
# exemplo de distribuição pra 2 estações diferentes
plt.subplot(3,1,1)
plt.title("mean users per day")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_bytes']]['users'].plot.bar(color='r', label = f"BS: {bs_info['most_bytes']}")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_packets']]['users'].plot.bar(color='g', label = f"BS: {bs_info['most_packets']}")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_user_access']]['users'].plot.bar(color='b', label = f"BS: {bs_info['most_user_access']}")
plt.legend()

plt.subplot(3,1,2)
plt.title("mean packets per day")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_bytes']]['packets'].plot.bar(color='r', label=f"BS: {bs_info['most_bytes']}")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_packets']]['packets'].plot.bar(color='g', label=f"BS: {bs_info['most_packets']}")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_user_access']]['packets'].plot.bar(color='b', label=f"BS: {bs_info['most_user_access']}")
plt.legend()

plt.subplot(3,1,3)
plt.title("mean bytes per day")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_bytes']]['bytes'].plot.bar(color='r', label=f"BS: {bs_info['most_bytes']}")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_packets']]['bytes'].plot.bar(color='g', label=f"BS: {bs_info['most_packets']}")
day_traffic_data[day_traffic_data.index.get_level_values('bs') == bs_info['most_user_access']]['bytes'].plot.bar(color='b', label=f"BS: {bs_info['most_user_access']}")
plt.legend()

plt.tight_layout()

## Per hour

In [ ]:
hour_traffic_data = cellular_traffic_data.groupby(['bs', 'hour']).mean()
hour_traffic_data

In [ ]:
max_hour = hour_traffic_data.idxmax()

We can see that, among all 24 hours, the BS with more users connected at the same hour was **BS 7846** at 8AM.

In [ ]:
hour_traffic_data[(hour_traffic_data.index.get_level_values('bs') == max_hour.users[0]) &
                 (hour_traffic_data.index.get_level_values('hour') == max_hour.users[1])]

The BS with more packets travelling was **BS 9286**, with a mean value of 251072 packets travelling at 2AM.

In [ ]:
hour_traffic_data[(hour_traffic_data.index.get_level_values('bs') == max_hour.packets[0]) &
                 (hour_traffic_data.index.get_level_values('hour') == max_hour.packets[1])]

The BS with the largest packages travelling per hour was **BS 10955** with 3.726652e+10 bytes at 9AM.

In [ ]:
hour_traffic_data[(hour_traffic_data.index.get_level_values('bs') == max_hour.bytes[0]) &
                 (hour_traffic_data.index.get_level_values('hour') == max_hour.bytes[1])]

In [ ]:
bs_hour_info = {
    "most_user_access": max_hour.users[0],
    "most_bytes": max_hour.bytes[0],
    "most_packets": max_hour.packets[0]
}

bs_hour_info

In [ ]:
plt.subplot(3,1,1)
plt.title(f"users")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_bytes']]['users'].plot(label=bs_hour_info['most_bytes'])
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_packets']]['users'].plot(label=bs_hour_info['most_packets'])
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_user_access']]['users'].plot(label=bs_hour_info['most_user_access'])
plt.legend()


plt.subplot(3,1,2)
plt.title(f"bytes")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_bytes']]['bytes'].plot(label=bs_hour_info['most_bytes'])
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_packets']]['bytes'].plot(label=bs_hour_info['most_packets'])
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_user_access']]['bytes'].plot(label=bs_hour_info['most_user_access'])
plt.legend()

plt.subplot(3,1,3)
plt.title(f"packets")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_bytes']]['packets'].plot(label=bs_hour_info['most_bytes'])
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_packets']]['packets'].plot(label=bs_hour_info['most_packets'])
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_user_access']]['packets'].plot(label=bs_hour_info['most_user_access'])

plt.legend()

plt.tight_layout()

In [ ]:
# exemplo de distribuição pra 2 estações diferentes
plt.subplot(3,1,1)
plt.title("mean users per day")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_bytes']]['users'].plot.bar(color='r', label = f"BS: {bs_hour_info['most_bytes']}")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_packets']]['users'].plot.bar(color='g', label = f"BS: {bs_hour_info['most_packets']}")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_user_access']]['users'].plot.bar(color='b', label = f"BS: {bs_hour_info['most_user_access']}")
plt.legend()

plt.subplot(3,1,2)
plt.title("mean packets per day")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_bytes']]['packets'].plot.bar(color='r', label=f"BS: {bs_hour_info['most_bytes']}")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_packets']]['packets'].plot.bar(color='g', label=f"BS: {bs_hour_info['most_packets']}")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_user_access']]['packets'].plot.bar(color='b', label=f"BS: {bs_hour_info['most_user_access']}")
plt.legend()

plt.subplot(3,1,3)
plt.title("mean bytes per day")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_bytes']]['bytes'].plot.bar(color='r', label=f"BS: {bs_hour_info['most_bytes']}")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_packets']]['bytes'].plot.bar(color='g', label=f"BS: {bs_hour_info['most_packets']}")
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == bs_hour_info['most_user_access']]['bytes'].plot.bar(color='b', label=f"BS: {bs_hour_info['most_user_access']}")
plt.legend()

plt.tight_layout()

In [ ]:
# exemplo de distribuição pra 2 estações diferente
plt.subplot(2,3,1)
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == 1]['users'].plot(kind='bar', title='users/day')
plt.subplot(2,3,2)
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == 1]['packets'].plot(kind='bar', title='packets/day')
plt.subplot(2,3,3)
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == 1]['bytes'].plot(kind='bar', title='bytes/day')

plt.subplot(2,3,4)
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == 298]['users'].plot(kind='bar', title='users/day')
plt.subplot(2,3,5)
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == 298]['packets'].plot(kind='bar', title='packets/day')
plt.subplot(2,3,6)
hour_traffic_data[hour_traffic_data.index.get_level_values('bs') == 298]['bytes'].plot(kind='bar', title='bytes/day')

plt.tight_layout()